In [1]:
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
import math
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
import numpy as np

## Run this if on windows

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Data loading

In [5]:
df=pd.read_csv('data.csv',header=None)

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,176,177,178,179,180,181,182,183,184,185
0,289,0,0,0,579,0,0,0,848,416,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1342,243,0,0,0,350,0,238,...,0,0,0,0,0,0,0,0,0,0
2,0,402,0,494,0,0,0,0,1802,472,...,0,0,0,0,0,0,0,0,0,0
3,0,309,1104,0,136,0,0,0,597,0,...,0,0,0,0,0,0,0,0,0,0
4,0,148,0,0,740,0,0,460,0,132,...,0,0,0,0,0,0,0,0,0,0


## Pipeline

In [17]:
class pipeline:
    def __init__(self,df):
        df=df.sample(frac=1)
        y=df.iloc[:,-36:]
        y=y.to_numpy()
        self.ds_size=len(y)
        self.train_ratio=0.7
        z=[]
        for i in y:
            z.append(str(i))
        self.le = preprocessing.LabelEncoder()
        #self.Y=tf.convert_to_tensor(self.le.fit_transform(z),dtype=tf.float32)
        self.Y=tf.convert_to_tensor(to_categorical(self.le.fit_transform(z)),dtype=tf.float32)
        #self.Y=tf.constant(le.fit_transform(z),dtype=tf.float32)
        self.X=df.iloc[:,:-36]
        #self.ds_train = tf.data.Dataset.from_tensor_slices((tf.constant(self.X.to_numpy().reshape(-1,50,3),dtype=tf.float32), self.Y))
        self.ds_data = tf.data.Dataset.from_tensor_slices((tf.constant(self.X.to_numpy(),dtype=tf.float32), self.Y))
        #self.combine_label_countMatrix(X,Y)
        #self.ds_data=self.ds_data.shuffle(1)
        #self.ds_data=self.ds_data.shuffle(5)
        #self.ds_data=self.ds_data.shuffle(8)
        #self.ds_data=self.ds_data.shuffle(2)
        #self.ds_data=self.ds_data.shuffle(3)
        #self.ds_data=self.ds_data.shuffle(33)
    
    def train_test(self):
        self.ds_train=self.ds_data.take(math.floor(self.ds_size*self.train_ratio))
        self.ds_test=self.ds_data.skip(math.floor(self.ds_size*self.train_ratio))
        self.ds_train.prefetch(self.ds_size-math.floor(self.ds_size*self.train_ratio))
        self.ds_test.prefetch(self.ds_size-math.floor(self.ds_size*self.train_ratio))

    def batching(self,BATCH_SIZE=16):
        self.train_test()
        self.ds_train_batched=self.ds_train.shuffle(10, reshuffle_each_iteration=True).batch(BATCH_SIZE).cache().shuffle(10, reshuffle_each_iteration=True).prefetch(tf.data.experimental.AUTOTUNE) 
        self.ds_test_batched=self.ds_test.batch(BATCH_SIZE).cache().prefetch(tf.data.experimental.AUTOTUNE) 
        return self.ds_train_batched,self.ds_test_batched

In [18]:
p=pipeline(df)

In [19]:
d=p.batching(8)

In [20]:
d

(<PrefetchDataset shapes: ((None, 150), (None, 21)), types: (tf.float32, tf.float32)>,
 <PrefetchDataset shapes: ((None, 150), (None, 21)), types: (tf.float32, tf.float32)>)

In [166]:
for i in d[0]:
    print(i[0])
    print(i[1])
    break

tf.Tensor(
[[ 595.  304.  588. ...    0.  494.    0.]
 [ 789.    0.    0. ...  345.    0.    0.]
 [ 427.    0.    0. ...  636.    0.    0.]
 ...
 [   0.    0.  202. ...    0.  525.    0.]
 [ 290.    0.    0. ...    0.  874.    0.]
 [   0.  566.    0. ...    0.    0. 1102.]], shape=(8, 150), dtype=float32)
tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]], shape=(8, 21), dtype=float32)


In [167]:
d[0]

<PrefetchDataset shapes: ((None, 150), (None, 21)), types: (tf.float32, tf.float32)>

## Model

In [263]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Reshape((50,3,1), input_shape=(150,)))
#model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Conv2D(5, (1, 3), strides=(1, 1),activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
#model.add(tf.keras.layers.Reshape((50,50,1)))
#model.add(tf.keras.layers.MaxPool2D((1,50)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(128,activation=tf.keras.layers.LeakyReLU(alpha=0.1),kernel_initializer='ones',
                              kernel_regularizer=tf.keras.regularizers.l1(0.01),
                              activity_regularizer=tf.keras.regularizers.l2(0.01)))
#model.add(tf.keras.layers.Dropout(0.3))
#model.add(tf.keras.layers.Dense(64,activation='sigmoid'))
#model.add(tf.keras.layers.Dropout(0.3))
#model.add(tf.keras.layers.Dense(64,activation='sigmoid'))
#model.add(tf.keras.layers.Dropout(0.3))
#model.add(tf.keras.layers.Dense(32,activation='sigmoid'))
#model.add(tf.keras.layers.Dropout(0.3))
#model.add(tf.keras.layers.Dense(32,activation='sigmoid'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(21,activation='softmax'))
#model = tf.keras.Model(inputs=x_i, outputs=x)

In [264]:
model.output_shape

(None, 21)

In [265]:
model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_15 (Reshape)         (None, 50, 3, 1)          0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 50, 1, 5)          20        
_________________________________________________________________
flatten_15 (Flatten)         (None, 250)               0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 250)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 128)               32128     
_________________________________________________________________
dropout_31 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 21)              

In [266]:
#model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(0.1),metrics=['accuracy'])
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.RMSprop(0.1),metrics=['accuracy'])

In [267]:
callbacks = [ReduceLROnPlateau(monitor='val_loss', patience=5, verbose=1, factor=0.3),
             ModelCheckpoint("model501.h5", save_best_only=True, monitor='val_loss'),
             ModelCheckpoint("model50.h5", save_best_only=True, monitor='val_accuracy'),
             EarlyStopping("val_loss", restore_best_weights = True, patience = 10)]

In [268]:
model.fit(d[0],validation_data=d[1],epochs=1000,verbose=1,callbacks=callbacks)

Epoch 1/1000
88/88 [==============================] - 1s 10ms/step - loss: 5820794.5000 - accuracy: 0.0470 - val_loss: 71696.8594 - val_accuracy: 0.0399
Epoch 2/1000
88/88 [==============================] - 1s 8ms/step - loss: 19816.4727 - accuracy: 0.0442 - val_loss: 8557.9248 - val_accuracy: 0.0432
Epoch 3/1000
88/88 [==============================] - 1s 8ms/step - loss: 51700.2383 - accuracy: 0.0613 - val_loss: 3116.6902 - val_accuracy: 0.0565
Epoch 4/1000
88/88 [==============================] - 1s 8ms/step - loss: 11646.8096 - accuracy: 0.0584 - val_loss: 15389.1475 - val_accuracy: 0.0399
Epoch 5/1000
88/88 [==============================] - 1s 8ms/step - loss: 9722.2490 - accuracy: 0.0798 - val_loss: 2690.0562 - val_accuracy: 0.0498
Epoch 6/1000
88/88 [==============================] - 1s 8ms/step - loss: 17745.6543 - accuracy: 0.0698 - val_loss: 2126.8677 - val_accuracy: 0.0664
Epoch 7/1000
88/88 [==============================] - 1s 8ms/step - loss: 15172.2822 - accuracy: 0.082

88/88 [==============================] - 1s 8ms/step - loss: 4.9095 - accuracy: 0.4330 - val_loss: 4.2861 - val_accuracy: 0.5681
Epoch 111/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.8989 - accuracy: 0.4402 - val_loss: 4.2790 - val_accuracy: 0.5615
Epoch 112/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.8771 - accuracy: 0.4046 - val_loss: 4.2862 - val_accuracy: 0.5947
Epoch 113/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.8043 - accuracy: 0.4359 - val_loss: 4.2858 - val_accuracy: 0.6246
Epoch 114/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.8920 - accuracy: 0.4003 - val_loss: 4.2528 - val_accuracy: 0.5914
Epoch 115/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.8328 - accuracy: 0.4160 - val_loss: 4.2291 - val_accuracy: 0.6080
Epoch 116/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.7297 - accuracy: 0.4145 - val_loss: 4.2576 - val_accuracy: 0.5814
Epoch 1

Epoch 167/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.5738 - accuracy: 0.4715 - val_loss: 4.0004 - val_accuracy: 0.5980
Epoch 168/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.5859 - accuracy: 0.4345 - val_loss: 4.0196 - val_accuracy: 0.6179
Epoch 169/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.4684 - accuracy: 0.4145 - val_loss: 3.9650 - val_accuracy: 0.5781
Epoch 170/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.5054 - accuracy: 0.4459 - val_loss: 3.9561 - val_accuracy: 0.6013
Epoch 171/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.5371 - accuracy: 0.4530 - val_loss: 3.9255 - val_accuracy: 0.6013
Epoch 172/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.5134 - accuracy: 0.4573 - val_loss: 3.9360 - val_accuracy: 0.6379
Epoch 173/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.5222 - accuracy: 0.4601 - val_loss: 3.9533 - val_accuracy:

Epoch 223/1000
88/88 [==============================] - 1s 7ms/step - loss: 4.1542 - accuracy: 0.5128 - val_loss: 3.6630 - val_accuracy: 0.6744
Epoch 224/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.0853 - accuracy: 0.4786 - val_loss: 3.6653 - val_accuracy: 0.6877
Epoch 225/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.1982 - accuracy: 0.5157 - val_loss: 3.6664 - val_accuracy: 0.6944
Epoch 226/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.1395 - accuracy: 0.4972 - val_loss: 3.6660 - val_accuracy: 0.6645
Epoch 227/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.1970 - accuracy: 0.4843 - val_loss: 3.6571 - val_accuracy: 0.6777
Epoch 228/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.1220 - accuracy: 0.4986 - val_loss: 3.6606 - val_accuracy: 0.6744
Epoch 229/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.1786 - accuracy: 0.4872 - val_loss: 3.6654 - val_accuracy:

Epoch 279/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.2045 - accuracy: 0.4829 - val_loss: 3.6409 - val_accuracy: 0.6744
Epoch 280/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.1094 - accuracy: 0.4972 - val_loss: 3.6414 - val_accuracy: 0.6678
Epoch 281/1000
88/88 [==============================] - 1s 8ms/step - loss: 4.0750 - accuracy: 0.5142 - val_loss: 3.6412 - val_accuracy: 0.6678
Epoch 282/1000
88/88 [==============================] - 1s 7ms/step - loss: 4.0941 - accuracy: 0.4872 - val_loss: 3.6412 - val_accuracy: 0.6678
Epoch 283/1000
88/88 [==============================] - 1s 7ms/step - loss: 4.1681 - accuracy: 0.4872 - val_loss: 3.6404 - val_accuracy: 0.6678

Epoch 00283: ReduceLROnPlateau reducing learning rate to 5.904900035602623e-07.
Epoch 284/1000
88/88 [==============================] - 1s 7ms/step - loss: 4.0638 - accuracy: 0.5100 - val_loss: 3.6400 - val_accuracy: 0.6678
Epoch 285/1000
88/88 [==============================] -

In [21]:
model=tf.keras.models.load_model("./model50.h5")

In [22]:
model.evaluate(d[0])

88/88 [==============================] - 0s 2ms/step - loss: 36.6386 - accuracy: 0.7407


[36.63861846923828, 0.7407407164573669]

In [23]:
model.evaluate(d[1])

38/38 [==============================] - 0s 3ms/step - loss: 24.0145 - accuracy: 0.7940


[24.014507293701172, 0.7940199375152588]

In [272]:
model=tf.keras.models.load_model("./model501.h5")

In [273]:
model.evaluate(d[0])

88/88 [==============================] - 0s 3ms/step - loss: 3.3927 - accuracy: 0.8262


[3.392746686935425, 0.8262107968330383]

In [274]:
model.evaluate(d[1])

38/38 [==============================] - 0s 3ms/step - loss: 3.6386 - accuracy: 0.6678


[3.6386280059814453, 0.6677740812301636]

In [24]:
model=tf.keras.models.load_model("./model69-82.h5")

In [25]:
model.evaluate(d[0])

88/88 [==============================] - 0s 2ms/step - loss: 3.5514 - accuracy: 0.7764


[3.5513830184936523, 0.7763532996177673]

In [26]:
model.evaluate(d[1])

38/38 [==============================] - 0s 2ms/step - loss: 3.3495 - accuracy: 0.8073


[3.349546194076538, 0.8073089718818665]

In [278]:
ypred=model.predict(d[1])

In [279]:
num=0
for i in range(0,15):
    print(num,":",np.argmax(ypred[i]))
    num=num+1

0 : 20
1 : 18
2 : 2
3 : 18
4 : 18
5 : 15
6 : 8
7 : 16
8 : 0
9 : 2
10 : 6
11 : 18
12 : 9
13 : 16
14 : 7


In [280]:
num=0
for i in d[1]:
    for j in i[1]:
        print(num,":",tf.math.argmax(j))
        num=num+1

0 : tf.Tensor(20, shape=(), dtype=int64)
1 : tf.Tensor(20, shape=(), dtype=int64)
2 : tf.Tensor(2, shape=(), dtype=int64)
3 : tf.Tensor(18, shape=(), dtype=int64)
4 : tf.Tensor(18, shape=(), dtype=int64)
5 : tf.Tensor(15, shape=(), dtype=int64)
6 : tf.Tensor(8, shape=(), dtype=int64)
7 : tf.Tensor(16, shape=(), dtype=int64)
8 : tf.Tensor(17, shape=(), dtype=int64)
9 : tf.Tensor(2, shape=(), dtype=int64)
10 : tf.Tensor(6, shape=(), dtype=int64)
11 : tf.Tensor(18, shape=(), dtype=int64)
12 : tf.Tensor(9, shape=(), dtype=int64)
13 : tf.Tensor(16, shape=(), dtype=int64)
14 : tf.Tensor(8, shape=(), dtype=int64)
15 : tf.Tensor(1, shape=(), dtype=int64)
16 : tf.Tensor(4, shape=(), dtype=int64)
17 : tf.Tensor(9, shape=(), dtype=int64)
18 : tf.Tensor(3, shape=(), dtype=int64)
19 : tf.Tensor(3, shape=(), dtype=int64)
20 : tf.Tensor(5, shape=(), dtype=int64)
21 : tf.Tensor(10, shape=(), dtype=int64)
22 : tf.Tensor(6, shape=(), dtype=int64)
23 : tf.Tensor(12, shape=(), dtype=int64)
24 : tf.Tensor(

In [183]:
for i in d[1]:
    for j in i[1]:
        print(j)

tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.

tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(21,), dtype=float32)
tf.Tensor([0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.

In [142]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_6 (Reshape)          (None, 50, 3, 1)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 50, 1, 5)          20        
_________________________________________________________________
flatten_6 (Flatten)          (None, 250)               0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 250)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               32128     
_________________________________________________________________
dropout_9 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 21)               

In [32]:
df1.head(50).to_numpy().flatten().reshape(1,-1).astype(np.int)

array([[   0,   51,   28,    7,    4,    1,   35,   54,    0,    0,   94,
           0,    0,    0,   28,    0,    0,    2,  434,    0,  564,    0,
           0,   47,    0,  326,  179,    0,    4,    0,    0,    0,  394,
           0,  116,    0,  520,  849,    0,    0,    7,    1,    0,    0,
         434,   14,    0,    0,    0,  234,  420,    0,    0,    8,    0,
           0,  195,    0,    0,    1,    0,  241,   59,    0,   50,    0,
          38,  149,    0,    0,    6,    0,    0,  759,    0,    0,    5,
           0,    0,   46,    0,    0,   45,   35,    0,   11,    0,    0,
           0,   94,    0,  110,    0,    0,    0,  213,    0,    0,   48,
          57,   25,   63,    0,    0,   73,    0,   50,   59,    0,    0,
         247,    0,  538,  619,    0,    6,    0,    0,  321,    0,    0,
          63,    0,    0,   32,   11,  946, 3354, 3157,  483, 2028, 4421,
           0,    0,   13,  109,    0,    0,   81,  884,    0,    0,  354,
           0,  477,    0,  140,    0, 

In [33]:
df1= pd.read_csv("a.csv",header=None)
df1=df1.iloc[1:]
df1=df1.loc[~(df1=='0').all(axis=1)]
df1.head(50).to_numpy().flatten()
prediction=model.predict(df1.head(50).to_numpy().flatten().reshape(1,-1).astype(np.int))

In [43]:
score=[]
prob=[]
l=math.floor(len(df1))
for i in range(0,l,50):
    if(i+50<=l):
        prediction=model.predict(df1.head(50).to_numpy().flatten().reshape(1,-1).astype(np.int))
        score=score+[prediction.tolist()[0].index(max(prediction.tolist()[0]))]
        prob=prob+[max(prediction.tolist()[0])]

In [44]:
score

[11, 11, 11, 11, 11]

In [45]:
prob

[0.22437740862369537,
 0.22437740862369537,
 0.22437740862369537,
 0.22437740862369537,
 0.22437740862369537]

In [38]:
prediction

array([[9.9106738e-04, 5.8418524e-04, 8.4574667e-06, 2.0647768e-08,
        2.5885119e-04, 1.5139569e-01, 9.4507411e-03, 2.3406519e-02,
        3.7662376e-02, 1.1018358e-01, 2.1039747e-01, 2.2437741e-01,
        1.8048068e-03, 1.2768515e-07, 1.0840325e-03, 1.0431050e-02,
        1.3121411e-01, 1.3703454e-03, 2.1795979e-02, 3.2057423e-02,
        3.1525765e-02]], dtype=float32)

In [42]:
p.le.inverse_transform([15])

array(['[0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]'],
      dtype='<U73')